In [1]:
import numpy as np

from market_env import MarketEnv
from market_model_builder import MarketModelBuilder

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']


In [4]:
import numpy as np

from market_env import MarketEnv
from market_model_builder import MarketModelBuilder
import tensorflow as tf

import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

#with tf.device('/cpu:0'):
#    filterbanks = tf.nn.conv2d(
#      filterbanks, delta_filter_stack, [1, 1, 1, 1], "SAME", data_format="NHWC",
#      name=name)    
    
class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        #initialize max memory and discount rate
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount
    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        #append states to the memory, if memory is over max delete earliest memory
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=30):
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1]
        inputs = []

        dim = len(self.memory[0][0][0])
        for i in range(dim):
            inputs.append([])
            
        targets = np.zeros((min(len_memory, batch_size), num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory, size=min(len_memory, batch_size))):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]
            
            for j in range(dim):
                inputs[j].append(state_t[j][0])
            
            #inputs.append(state_t)
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            #with tf.device('/cpu:0'):
            targets[i] = model.predict(state_t)[0]
            Q_sa = np.max(model.predict(state_tp1)[0])
            if game_over:  # if game_over is True
                #with tf.device('/cpu:0'):
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                #with tf.device('/cpu:0'):
                targets[i, action_t] = reward_t + self.discount * Q_sa
        
        #inputs = np.array(inputs)
        inputs = [np.array(inputs[i]) for i in range(dim)]
        #print(inputs)
        #print(reward_t)
        #print(action_t)
        #print(i)

        return inputs, targets
    
    #def reset(self):
    #return env.reset()

if __name__ == "__main__":
    import sys
    import codecs

    #codeListFilename = sys.argv[1]
    #modelFilename = sys.argv[2] if len(sys.argv) > 2 else None
    codeListFilename = "./SSE_10.csv"
    modelFilename = "./model.h5"
    #modelFilename = sys.argv[2] if len(sys.argv) > 2 else None
    
    #print(codeListFilename)
    codeMap = {}
    f = codecs.open(codeListFilename, "r", "utf-8")

    for line in f:
        if line.strip() != "":
            tokens = line.strip().split(",") if not "\t" in line else line.strip().split("\t")
            #codeMap[tokens[0]] = tokens[1]
            codeMap[tokens[0]] = tokens[0]

    f.close()
    #print(codeMap.keys())
    env = MarketEnv(dir_path = "./data", target_codes = list(codeMap), input_codes = [], start_date = "2013-06-10", end_date = "2020-10-30", sudden_death = -1.0)
    #print(env)
    # parameters
    epsilon = 0.3213  # exploration
    min_epsilon = 0.1
    epoch = 10
    max_memory = 100
    batch_size = 32
    discount = 0.8

    from keras.optimizers import SGD
    #with tf.device('/device:GPU:0'):
    model = MarketModelBuilder(modelFilename).getModel()
    #with tf.device('/device:GPU:0'):
    sgd = SGD(lr = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
    #with tf.device('/device:GPU:0'):
    model.compile(loss='mse', optimizer='rmsprop')

    # Initialize experience replay object
    exp_replay = ExperienceReplay(max_memory = max_memory, discount = discount)

    # Train
    win_cnt = 0
    for e in range(epoch):
        loss = 0.
        env._reset()
        game_over = False
        # get initial input
        input_t = env._reset()
        #input_t = [input_t[0], np.reshape(input_t[1], (1, 60, 2, 1))]
        cumReward = 0

        while not game_over:
            input_tm1 = input_t
            isRandom = False

            # get next action
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, env.action_space.n, size=1)[0]

                isRandom = True
            else:
                #with tf.device('/device:GPU:0'):
                q = model.predict(input_tm1)
                #print(q)
                action = np.argmax(q[0])

                #print "  ".join(["%s:%.2f" % (l, i) for l, i in zip(env.actions, q[0].tolist())])
                if np.nan in q:
                    print("OCCUR NaN!!!")
                    exit()

            # apply action, get rewards and new state
            input_t, reward, game_over, info = env._step(action)
            #input_t = [input_t[0], np.reshape(input_t[1], (1, 60, 2, 1))]
            cumReward += reward

            if env.actions[action] == "LONG" or env.actions[action] == "SHORT":
                color = bcolors.FAIL if env.actions[action] == "LONG" else bcolors.OKBLUE
                if isRandom:
                    color = bcolors.WARNING if env.actions[action] == "LONG" else bcolors.OKGREEN
            print("%s:\t%s\t%.2f\t%.2f\t" % (info["dt"], color + env.actions[action] + bcolors.ENDC, cumReward, info["cum"]) + ("\t".join(["%s:%.2f" % (l, i) for l, i in zip(env.actions, q[0].tolist())]) if isRandom == False else ""))

            # store experience
            exp_replay.remember([input_tm1, action, reward, input_t], game_over)

            # adapt model
            #with tf.device('/device:GPU:0'):
            inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)
            with tf.device('/device:GPU:0'):
                loss += model.train_on_batch(inputs, targets)

        if cumReward > 0 and game_over:
            win_cnt += 1

        print("Epoch {:03d}/{} | Loss {:.4f} | Win count {} | Epsilon {:.4f}".format(e, epoch, loss, win_cnt, epsilon))
        # Save trained model weights and architecture, this will be used by the visualization code
        model.save_weights("model.h5", overwrite=True)
        epsilon = max(min_epsilon, epsilon * 0.99)



2013-06-19:	LONG	0.00	1.42	LONG:1.00	SHORT:0.00
2013-06-20:	LONG	0.00	0.99	LONG:1.00	SHORT:0.00
2013-06-21:	LONG	0.00	0.86	
2013-06-24:	LONG	0.00	0.57	LONG:1.00	SHORT:0.00
2013-06-25:	LONG	0.00	0.40	LONG:1.00	SHORT:0.00
2013-06-26:	SHORT	-2.73	0.59	
2013-06-27:	SHORT	-2.73	0.46	
2013-06-28:	LONG	-2.85	0.40	LONG:1.00	SHORT:0.00
2013-07-01:	LONG	-2.85	0.04	LONG:1.00	SHORT:0.00
2013-07-02:	LONG	-2.85	0.21	LONG:1.00	SHORT:0.00
2013-07-03:	SHORT	-0.21	0.03	
2013-07-05:	LONG	0.65	0.27	
2013-07-08:	LONG	0.65	0.08	LONG:1.00	SHORT:0.00
2013-07-09:	SHORT	1.82	-0.00	LONG:0.00	SHORT:1.00
2013-07-10:	SHORT	1.82	-0.05	LONG:0.00	SHORT:1.00
2013-07-11:	LONG	1.82	-0.09	
2013-07-12:	SHORT	26.00	0.09	LONG:0.00	SHORT:1.00
2013-07-15:	SHORT	26.00	0.27	LONG:0.00	SHORT:1.00
2013-07-16:	SHORT	24.21	-0.02	LONG:0.00	SHORT:1.00
2013-07-17:	SHORT	24.21	-0.11	
2013-07-18:	SHORT	25.07	-0.25	LONG:0.00	SHORT:1.00
2013-07-19:	SHORT	24.37	0.02	LONG:0.00	SHORT:1.00
2013-07-22:	SHORT	24.37	0.83	LONG:0.00	SHORT:1.00
2013-

2014-01-31:	SHORT	176.00	-2.58	
2014-02-03:	SHORT	176.00	-3.10	
2014-02-04:	LONG	174.47	-2.21	
2014-02-05:	LONG	174.47	-2.30	
2014-02-06:	SHORT	174.25	-2.18	LONG:0.00	SHORT:1.00
2014-02-07:	SHORT	174.25	-2.30	LONG:0.00	SHORT:1.00
2014-02-10:	SHORT	174.25	-2.45	LONG:0.00	SHORT:1.00
2014-02-11:	SHORT	174.25	-2.46	LONG:0.00	SHORT:1.00
2014-02-12:	SHORT	174.25	-2.57	LONG:0.00	SHORT:1.00
2014-02-13:	SHORT	174.25	-2.56	LONG:0.00	SHORT:1.00
2014-02-14:	SHORT	174.25	-2.67	
2014-02-18:	SHORT	174.25	-2.66	LONG:0.00	SHORT:1.00
2014-02-19:	LONG	173.54	-2.57	
2014-02-20:	SHORT	173.50	-2.78	LONG:0.00	SHORT:1.00
2014-02-21:	SHORT	173.50	-2.82	LONG:0.00	SHORT:1.00
2014-02-24:	LONG	173.39	-2.79	
2014-02-25:	SHORT	173.38	-2.84	LONG:0.00	SHORT:1.00
2014-02-26:	SHORT	173.38	-2.33	LONG:0.00	SHORT:1.00
2014-02-27:	SHORT	173.38	-2.51	LONG:0.00	SHORT:1.00
2014-02-28:	SHORT	173.38	-2.32	LONG:0.00	SHORT:1.00
2014-03-03:	SHORT	173.38	-2.23	
2014-03-04:	SHORT	173.38	-2.42	LONG:0.00	SHORT:1.00
2014-03-05:	LONG	173

2014-09-10:	SHORT	170.27	-3.87	
2014-09-11:	SHORT	170.27	-3.85	LONG:0.00	SHORT:1.00
2014-09-12:	SHORT	170.27	-3.82	
2014-09-15:	SHORT	170.27	-4.20	LONG:0.00	SHORT:1.00
2014-09-16:	SHORT	170.27	-4.54	LONG:0.00	SHORT:1.00
2014-09-17:	LONG	169.11	-4.58	
2014-09-18:	SHORT	169.12	-4.54	LONG:0.00	SHORT:1.00
2014-09-19:	SHORT	169.12	-4.60	LONG:0.00	SHORT:1.00
2014-09-22:	SHORT	169.12	-4.94	
2014-09-23:	SHORT	169.12	-4.82	
2014-09-24:	SHORT	169.12	-4.85	LONG:0.00	SHORT:1.00
2014-09-25:	SHORT	169.12	-4.94	LONG:0.00	SHORT:1.00
2014-09-26:	SHORT	169.12	-5.15	LONG:0.00	SHORT:1.00
2014-09-29:	SHORT	169.12	-5.12	LONG:0.00	SHORT:1.00
2014-09-30:	SHORT	169.12	-5.24	LONG:0.00	SHORT:1.00
2014-10-01:	SHORT	169.12	-5.61	LONG:0.00	SHORT:1.00
2014-10-02:	SHORT	169.12	-5.77	LONG:0.00	SHORT:1.00
2014-10-03:	SHORT	169.12	-5.72	
2014-10-06:	SHORT	169.12	-5.47	LONG:0.00	SHORT:1.00
2014-10-07:	LONG	168.05	-5.48	
2014-10-08:	SHORT	168.05	-5.58	LONG:0.00	SHORT:1.00
2014-10-09:	SHORT	168.05	-5.70	LONG:0.00	SHORT:1.0

2015-04-17:	SHORT	170.74	-8.85	
2015-04-20:	SHORT	170.74	-9.25	LONG:0.00	SHORT:1.00
2015-04-21:	SHORT	170.74	-9.16	
2015-04-22:	LONG	170.44	-9.37	
2015-04-23:	SHORT	170.46	-9.46	
2015-04-24:	SHORT	170.46	-9.53	
2015-04-27:	SHORT	170.46	-9.35	LONG:0.00	SHORT:1.00
2015-04-28:	SHORT	170.46	-9.07	LONG:0.00	SHORT:1.00
2015-04-29:	SHORT	170.46	-8.85	LONG:0.00	SHORT:1.00
2015-04-30:	SHORT	170.46	-9.13	LONG:0.00	SHORT:1.00
2015-05-01:	SHORT	170.46	-9.32	LONG:0.00	SHORT:1.00
2015-05-04:	LONG	170.39	-9.49	
2015-05-05:	SHORT	170.41	-9.52	LONG:0.00	SHORT:1.00
2015-05-06:	SHORT	170.41	-8.95	LONG:0.00	SHORT:1.00
2015-05-07:	SHORT	170.41	-9.87	LONG:0.00	SHORT:1.00
2015-05-08:	SHORT	170.41	-9.66	LONG:0.00	SHORT:1.00
2015-05-11:	LONG	170.28	-9.89	
2015-05-12:	SHORT	170.31	-9.79	LONG:0.00	SHORT:1.00
2015-05-13:	SHORT	170.31	-9.75	LONG:0.00	SHORT:1.00
2015-05-14:	SHORT	170.31	-10.08	LONG:0.00	SHORT:1.00
2015-05-15:	SHORT	170.31	-9.81	LONG:0.00	SHORT:1.00
2015-05-18:	SHORT	170.31	-9.72	LONG:0.00	SHORT:1.0

2015-11-17:	SHORT	170.03	-12.58	
2015-11-18:	LONG	170.10	-12.71	
2015-11-19:	SHORT	170.11	-12.60	LONG:0.00	SHORT:1.00
2015-11-20:	SHORT	170.11	-12.33	LONG:0.00	SHORT:1.00
2015-11-23:	LONG	170.16	-12.34	
2015-11-24:	SHORT	170.16	-12.47	LONG:0.00	SHORT:1.00
2015-11-25:	LONG	170.15	-12.49	
2015-11-27:	SHORT	170.15	-12.48	
2015-11-30:	SHORT	170.15	-12.63	LONG:0.00	SHORT:1.00
2015-12-01:	SHORT	170.15	-12.15	LONG:0.00	SHORT:1.00
2015-12-02:	SHORT	170.15	-12.13	LONG:0.00	SHORT:1.00
2015-12-03:	SHORT	170.15	-11.99	LONG:0.00	SHORT:1.00
2015-12-04:	SHORT	170.15	-11.99	
2015-12-07:	SHORT	170.15	-12.23	
2015-12-08:	SHORT	170.15	-12.02	LONG:0.00	SHORT:1.00
2015-12-09:	SHORT	170.15	-12.13	LONG:0.00	SHORT:1.00
2015-12-10:	LONG	170.22	-12.24	
2015-12-11:	LONG	170.22	-12.10	
2015-12-14:	SHORT	170.21	-12.19	LONG:0.00	SHORT:1.00
2015-12-15:	LONG	170.20	-12.48	
2015-12-16:	SHORT	170.22	-13.18	LONG:0.00	SHORT:1.00
2015-12-17:	SHORT	170.22	-12.84	LONG:0.00	SHORT:1.00
2015-12-18:	LONG	170.22	-12.61	
2015-12-

...

2018-05-03:	SHORT	23601.64	-20.28	LONG:0.00	SHORT:1.00
2018-05-04:	SHORT	23601.64	-20.23	LONG:0.00	SHORT:1.00
2018-05-07:	SHORT	23601.64	-20.51	LONG:0.00	SHORT:1.00
2018-05-08:	SHORT	23601.64	-20.33	LONG:0.00	SHORT:1.00
2018-05-09:	SHORT	23601.64	-20.43	LONG:0.00	SHORT:1.00
2018-05-10:	SHORT	23601.64	-20.57	LONG:0.00	SHORT:1.00
2018-05-11:	SHORT	23601.64	-20.79	
2018-05-14:	SHORT	23601.64	-20.69	LONG:0.00	SHORT:1.00
2018-05-15:	SHORT	23601.64	-20.99	
2018-05-16:	LONG	23601.39	-20.97	
2018-05-17:	SHORT	23601.39	-21.06	LONG:0.00	SHORT:1.00
2018-05-18:	SHORT	23601.39	-21.24	LONG:0.00	SHORT:1.00
2018-05-21:	SHORT	23601.39	-21.36	LONG:0.00	SHORT:1.00
2018-05-22:	SHORT	23601.39	-21.19	LONG:0.00	SHORT:1.00
2018-05-23:	SHORT	23601.39	-21.13	LONG:0.00	SHORT:1.00
2018-05-24:	SHORT	23601.39	-21.57	LONG:0.00	SHORT:1.00
2018-05-25:	SHORT	23601.39	-21.41	LONG:0.00	SHORT:1.00
2018-05-29:	SHORT	23601.39	-21.20	LONG:0.00	SHORT:1.00
2018-05-30:	SHORT	23601.39	-21.21	
2018-05-31:	SHORT	23601.39	-21.33	LO

2018-11-23:	LONG	23601.84	-15.07	
2018-11-26:	LONG	23601.84	-15.13	
2018-11-27:	SHORT	23601.85	-15.07	LONG:0.00	SHORT:1.00
2018-11-28:	SHORT	23601.85	-14.97	
2018-11-29:	SHORT	23601.85	-14.45	LONG:0.00	SHORT:1.00
2018-11-30:	SHORT	23601.85	-15.01	LONG:0.00	SHORT:1.00
2018-12-03:	SHORT	23601.85	-14.64	LONG:0.00	SHORT:1.00
2018-12-04:	LONG	23601.93	-14.71	
2018-12-06:	LONG	23601.93	-14.70	
2018-12-07:	SHORT	23601.94	-14.96	LONG:0.00	SHORT:1.00
2018-12-10:	SHORT	23601.94	-14.57	LONG:0.00	SHORT:1.00
2018-12-11:	SHORT	23601.94	-14.44	
2018-12-12:	SHORT	23601.94	-14.37	LONG:0.00	SHORT:1.00
2018-12-13:	LONG	23602.02	-13.82	
2018-12-14:	SHORT	23601.98	-13.80	LONG:0.00	SHORT:1.00
2018-12-17:	SHORT	23601.98	-13.83	LONG:0.00	SHORT:1.00
2018-12-18:	SHORT	23601.98	-13.76	LONG:0.00	SHORT:1.00
2018-12-19:	SHORT	23601.98	-14.01	LONG:0.00	SHORT:1.00
2018-12-20:	SHORT	23601.98	-13.94	LONG:0.00	SHORT:1.00
2018-12-21:	SHORT	23601.98	-13.98	LONG:0.00	SHORT:1.00
2018-12-24:	SHORT	23601.98	-14.47	LONG:0.00	S

2019-06-17:	LONG	23604.46	-10.76	
2019-06-18:	SHORT	23604.50	-10.79	LONG:0.00	SHORT:1.00
2019-06-19:	SHORT	23604.50	-10.85	LONG:0.00	SHORT:1.00
2019-06-20:	SHORT	23604.50	-10.80	
2019-06-21:	SHORT	23604.50	-10.30	
2019-06-24:	SHORT	23604.50	-9.95	LONG:0.00	SHORT:1.00
2019-06-25:	SHORT	23604.50	-10.49	LONG:0.00	SHORT:1.00
2019-06-26:	LONG	23604.52	-10.04	
2019-06-27:	SHORT	23604.48	-10.01	LONG:0.00	SHORT:1.00
2019-06-28:	SHORT	23604.48	-10.31	LONG:0.00	SHORT:1.00
2019-07-01:	SHORT	23604.48	-10.26	LONG:0.00	SHORT:1.00
2019-07-02:	SHORT	23604.48	-9.63	LONG:0.00	SHORT:1.00
2019-07-03:	SHORT	23604.48	-9.57	LONG:0.00	SHORT:1.00
2019-07-05:	SHORT	23604.48	-9.53	LONG:0.00	SHORT:1.00
2019-07-08:	SHORT	23604.48	-10.03	LONG:0.00	SHORT:1.00
2019-07-09:	SHORT	23604.48	-10.32	LONG:0.00	SHORT:1.00
2019-07-10:	SHORT	23604.48	-10.30	
2019-07-11:	SHORT	23604.48	-9.89	LONG:0.00	SHORT:1.00
2019-07-12:	SHORT	23604.48	-9.48	LONG:0.00	SHORT:1.00
2019-07-15:	SHORT	23604.48	-8.89	
2019-07-16:	SHORT	23604.48	-1

2020-01-10:	SHORT	23601.55	-12.22	LONG:0.00	SHORT:1.00
2020-01-13:	SHORT	23601.55	-12.02	LONG:0.00	SHORT:1.00
2020-01-14:	SHORT	23601.55	-12.17	LONG:0.00	SHORT:1.00
2020-01-15:	SHORT	23601.55	-12.39	
2020-01-16:	SHORT	23601.55	-12.76	LONG:0.00	SHORT:1.00
2020-01-17:	SHORT	23601.55	-12.81	
2020-01-21:	SHORT	23601.55	-12.73	LONG:0.00	SHORT:1.00
2020-01-22:	LONG	23601.00	-12.84	
2020-01-23:	SHORT	23601.01	-12.88	
2020-01-24:	SHORT	23601.01	-12.73	
2020-01-27:	SHORT	23601.01	-13.19	LONG:0.00	SHORT:1.00
2020-01-28:	SHORT	23601.01	-13.11	LONG:0.00	SHORT:1.00
2020-01-29:	SHORT	23601.01	-13.24	LONG:0.00	SHORT:1.00
2020-01-30:	SHORT	23601.01	-13.60	
2020-01-31:	SHORT	23601.01	-13.54	LONG:0.00	SHORT:1.00
2020-02-03:	SHORT	23601.01	-13.82	LONG:0.00	SHORT:1.00
2020-02-04:	LONG	23600.61	-13.60	
2020-02-05:	SHORT	23600.59	-14.02	LONG:0.00	SHORT:1.00
2020-02-06:	SHORT	23600.59	-13.61	LONG:0.00	SHORT:1.00
2020-02-07:	SHORT	23600.59	-13.45	LONG:0.00	SHORT:1.00
2020-02-10:	LONG	23600.65	-13.86	
2020-02-

2020-08-11:	LONG	23598.15	-15.23	
2020-08-12:	SHORT	23598.09	-14.64	LONG:0.00	SHORT:1.00
2020-08-13:	SHORT	23598.09	-14.52	LONG:0.00	SHORT:1.00
2020-08-14:	LONG	23598.14	-14.88	
2020-08-17:	SHORT	23598.17	-15.11	LONG:0.00	SHORT:1.00
2020-08-18:	LONG	23598.15	-15.39	
2020-08-19:	LONG	23598.15	-15.61	
2020-08-20:	SHORT	23598.20	-16.02	LONG:0.00	SHORT:1.00
2020-08-21:	SHORT	23598.20	-16.25	LONG:0.00	SHORT:1.00
2020-08-24:	SHORT	23598.20	-15.83	LONG:0.00	SHORT:1.00
2020-08-25:	LONG	23598.22	-15.86	
2020-08-26:	SHORT	23598.23	-16.32	LONG:0.00	SHORT:1.00
2020-08-27:	SHORT	23598.23	-16.28	LONG:0.00	SHORT:1.00
2020-08-28:	SHORT	23598.23	-16.41	LONG:0.00	SHORT:1.00
2020-08-31:	SHORT	23598.23	-16.53	LONG:0.00	SHORT:1.00
2020-09-01:	SHORT	23598.23	-16.68	LONG:0.00	SHORT:1.00
2020-09-02:	LONG	23598.11	-16.69	
2020-09-03:	SHORT	23598.11	-16.81	LONG:0.00	SHORT:1.00
2020-09-04:	SHORT	23598.11	-16.56	LONG:0.00	SHORT:1.00
2020-09-08:	SHORT	23598.11	-16.45	LONG:0.00	SHORT:1.00
2020-09-09:	SHORT	23598.11

In [8]:
np.size(inputs[1])

150528

In [1]:
print(codeMap)

NameError: name 'codeMap' is not defined

In [3]:
print(target_codes)

NameError: name 'target_codes' is not defined